# Clasificación

**Autor: Andrés M. Castillo**

Objetivo: 

* Implementar el algoritmo de árbol de decisión ID3
* Preparar los datos, entrenar y evaluar el desempeño de un clasificador

## Árboles de decisión

Modelo de clasificación también conocido como ID3 que significa "inducción mediante árboles de decisión" que fue desarrollado por J. Ross Quinlan, capaz de tomar decisiones con gran precisión. Sistema de aprendizaje supervisado que aplica la estrategia "divide y vencerás" para hacer la clasificación, implementando métodos y técnicas para la realización de procesos inteligentes, representando así el conocimiento y el aprendizaje, con el propósito de automatizar tareas.

En este laboratorio, usted va a implementar el algoritmo llamado ID3, y lo evaluará usando el conjunto de datos de clasificación llamado Titanic, disponible en Kaggle [Titanic Kaggle](https://www.kaggle.com/c/titanic/)

# Paso 1. Importar las librerías


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import sklearn
from sklearn import tree

# Paso 2. Importar el conjunto de datos

In [49]:
trainx=pd.read_csv("./data/train.csv")
trainx.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [50]:
print(trainx.shape)

(891, 12)


# Paso 3. Preprocesamiento

Convertir los datos a nominales u ordinales para implementar el algoritmo ID3. Por ejemplo, usando la edad podemos crear un atributo que diga si el pasajero era niño o adulto:

In [51]:
train = trainx.copy()
train["Child"] = float('NaN')

# Assign 1 to passengers under 18, 0 to those 18 or older. Print the new column.
train.loc[train["Age"] < 18, "Child"] = 1
train.loc[train["Age"] >= 18, "Child"] = 0
train["Child"].head()

0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: Child, dtype: float64

Ahora que ya no necesitamos el atributo "Age", podemos removerlo del dataframe usando la función drop

In [52]:
train = train.drop(['Age'], axis = 1)

In [53]:
train.head(10)

,PassengerId,Survived,Pclass,Name,Sex,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Child
0,1,0,3,"Braund, Mr. Owen Harris",male,1,0,A/5 21171,7.2500,NaN,S,0.0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,1,0,PC 17599,71.2833,C85,C,0.0
2,3,1,3,"Heikkinen, Miss. Laina",female,0,0,STON/O2. 3101282,7.9250,NaN,S,0.0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,1,0,113803,53.1000,C123,S,0.0
4,5,0,3,"Allen, Mr. William Henry",male,0,0,373450,8.0500,NaN,S,0.0
5,6,0,3,"Moran, Mr. James",male,0,0,330877,8.4583,NaN,Q,NaN
6,7,0,1,"McCarthy, Mr. Timothy J",male,0,0,17463,51.8625,E46,S,0.0
7,8,0,3,"Palsson, Master. Gosta Leonard",male,3,1,349909,21.0750,NaN,S,1.0
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,0,2,347742,11.1333,NaN,S,0.0
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,1,0,237736,30.0708,NaN,C,1.0


**Tarea** Convierta el atributo Fare en un atributo ordinal de 3 niveles. 

Esta tarea no es tan trivial, ya que el atributo no se distribuye uniformemente en todo el rango de valores. Hay muchos valores pequeños y unos cuantos valores muy grandes. Por tanto es conveniente usar una escala logarítmica para antes de realizar la discretización. Luego, divida el intervalo comprendido entre el menor y el mayor valor numérico y divídalo en 3. Luego asigne a cada ejemplo un valor 0, 1 or 2 dependiendo del rango en el cual se encuentre. Luego elimine el atributo original Fare.

**Pista**
* Use las funciones `max()` y `min()` de de los arreglos de pandas.
* También será necesario sumar 1 a los valores originales antes de calcular el logaritmo, para evitar calcular el log(0)
* La operación de discretización se puede realizar con una simple operación algebráica. 
* Reasigne el resultado de la operación sobre la antigüa columna 'Fare'

In [54]:
# START CODE HERE
fare = np.log(train.Fare + 1)
minFare = fare.min()
maxFare = fare.max()
newFare = np.floor(2 * (fare - minFare) / (maxFare - minFare))

train['Fare'] = newFare
# START CODE HERE
print(train['Fare'])

0      0.0
1      1.0
2      0.0
3      1.0
4      0.0
      ... 
886    0.0
887    1.0
888    1.0
889    1.0
890    0.0
Name: Fare, Length: 891, dtype: float64


**Valor esperado**
```CPP
0      0.0
1      1.0
2      0.0
3      1.0
4      0.0
      ... 
886    0.0
887    1.0
888    1.0
889    1.0
890    0.0
Name: Fare, Length: 891, dtype: float64
```

# Paso 4. Selecciones los atributos que usará para el entrenamiento

Usando las operaciones de slicing, separe en otro data frame, solo los atributos que usará en el problema de entrenamiento. Mantenga el atributo **PassengerId** aunque este no debe usado como atributo descriptivo.

En este paso es buena idea, crear un nuevo dataframe, que contendrá la copia de los valores que se necesitan del dataframe original. Para eso puede usar la línea que muestra abajo. attributes, será una lista con los nombres de los atributos a mantener

In [57]:
# START CODE HERE
attributes = ['PassengerId', 'Survived', 'Pclass', 'Sex', 'SibSp', 'Parch', 'Fare', 'Embarked', 'Child']
# START CODE HERE
train2 = train.loc[:, attributes].copy()

# Paso 5. Elimine todos los ejemplo con valores nulos, inválidos o NaN

Antes de pasar al siguiente paso, debe asegurarse que todos los valores de su tabla continene valores válidos. Aunque hay varias formas de resolver este problema, como por ejemplo completando los faltantes, para este primer ejercicio simplemente elimine los valores problemáticos. 

Para esto use la función dropna() de pandas: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html

**Pista**

No olvide reasignar en train2 la salida de la función dropna()

In [60]:
print(train2.shape)
# START CODE HERE
train2 = train2.dropna()
# START CODE HERE
print(train2.shape)

(891, 9)
(712, 9)


**Valor esperado**
```
(891, 9)
(712, 9)
```

# Paso 6. Separe los datos en X y L

Haga una copia de los ejemplos solo con las columnas categóricas(ordinales y nominales) en un dataframe llamado **X**. Este dataframe no tendrá el *PassangerId* ni el nombre ni ningún otro atributo que no se vaya a usar para construir el árbol de inducción.
En otro dataframe **L**, de una sola columna, copie las etiquetas de los ejemplos. Es decir el atributo **Survived**

In [61]:
# START CODE HERE
X = train2.loc[:,['Pclass', 'Sex', 'SibSp', 'Parch', 'Fare', 'Embarked', 'Child']].copy()
L = train2.loc[:, 'Survived'].copy()
# START CODE HERE
print(X.shape)
print(L.shape)

(712, 7)
(712,)


**Valor esperado**

```
(712, 7)
(712,)
```

# Paso 7. Ganancia de información

Implemente la función Gain(data, column)

In [156]:
def entropy(values, labels):
    categories = np.unique(values)
    n_samples = len(values)
    p = np.sum(labels == 1)
    n = n_samples - p
    e = 0 # Store here the entropy
    
    for i in categories:
    # START CODE HERE
        labels_i = labels[values == i]
        pi = np.sum(labels_i == 1)
        ni = np.sum(labels_i == 0)
        e += (pi + ni) / (p + n) * info(pi, ni)
    # END CODE HERE
    return e

def info(p, n):
    if p * n == 0:
        return 0
    # START CODE HERE
    return - p / (p + n) * np.log2( p / (p + n)) - n / (p + n) * np.log2( n / (p + n))
    # END CODE HERE
    return 0

def gain(X, L, col_name):
    n_samples = len(X)
    p = np.sum(L == 1)
    n = n_samples - p
    # START CODE HERE
    return info(p, n) - entropy(X.loc[:, col_name], L)
    # END CODE HERE


Si las implementaciones son correctas las siguientes celdas deben darle los valores esperados

In [157]:
info(5, 10)

0.9182958340544896

**Valor esperado**

0.9182958340544896

In [158]:
entropy(train2.loc[:,'Child'], L)

0.9633563187521076

**Valor esperado**

0.036153948204607045

In [159]:
print(f"Pclass {gain(X, L, 'Pclass')}")
print(f"Sex {gain(X, L, 'Sex')}")
print(f"SibSp {gain(X, L, 'SibSp')}")
print(f"Parch {gain(X, L, 'Parch')}")
print(f"Fare {gain(X, L, 'Fare')}")
print(f"Embarked {gain(X, L, 'Embarked')}")
print(f"Child {gain(X, L, 'Child')}")

Pclass 0.09400998456880605
Sex 0.21410831283572307
SibSp 0.024868993114017135
Parch 0.030682011788098373
Fare 0.056844573680660315
Embarked 0.027858206925992057
Child 0.010162683632573333


**Valor esperado**
```
Pclass 0.09400998456880605
Sex 0.21410831283572307
SibSp 0.024868993114017135
Parch 0.030682011788098373
Fare 0.056844573680660315
Embarked 0.027858206925992057
Child 0.010162683632573333
```

***Pregunta: Según estos valores ¿Cúal es el atributo que debería usarse como regla de decisión en el primer nivel del árbol?***

# Paso 8. J45

Ahora implemente el algoritmo de construcción del árbol de inducción J45, que se describe en las diapositivas de la clase pasada

In [194]:
def j45level(X, L, nMin=20, minRatio=0.8, level = 0):
    '''
    Entradas:
        X: Conjunto de datos
        L: Etiquetas
    Salidas:
        arbol
    '''
    # Un truco para agregar tabulados dependiendo el nivel
    tabs = "".join((['    '] * level))
    attributes = X.columns
    
    # Contamos cuantos positivos y cuantos negativos tenemos
    pos_neg = (np.sum(L == 1), np.sum(L == 0))
    
    # Calcule el ratio entro la clase menos frecuente y la clase mas frecuente
    # START CODE HERE 
    currentRatio = np.min(pos_neg) / np.max(pos_neg)
    # END CODE HERE 

    # Esto es lo que hacemos para saber que llegamos a una hoja
    if len(attributes) == 0 or len(X) <= nMin or currentRatio < minRatio:
        if pos_neg[0] >= pos_neg[1]:
            print(f"{tabs}Positive {pos_neg}")
        else:
            print(f"{tabs}Negative {pos_neg}")
                
        return pos_neg
    
    gains = dict()
    for att in attributes:
        # START CODE HERE 
        gains[att] = gain(X, L, att)
        # END CODE HERE 
        
    sorted_x = sorted(gains.items(), key=lambda kv: kv[1], reverse=True)
    
    #Imprime el mejor atributo con su correspondiente ganancia de información
    print(f"{tabs}Best: {sorted_x[0]}")
    
    key = sorted_x[0][0]
    # Ahora partimos el conjunto según los valores del mejor atributo
    categories = np.unique(X.loc[:, key])
    for cat in categories:
        print(f"{tabs}  {cat}")
        # START CODE HERE 
        lines = X[key] == cat
        j45level(X.drop(key, axis=1).loc[lines, :], L.loc[lines], nMin, minRatio, level +  1)
        # END CODE HERE 
        
    print(f"{tabs}end of {key}")
  


In [195]:
j45level(X, L, 100, 0.2)

Best: ('Sex', 0.21410831283572307)
  female
    Best: ('Pclass', 0.2267010508317432)
      1
        Positive (80, 3)
      2
        Positive (68, 6)
      3
        Best: ('Fare', 0.07516755751248805)
          0.0
            Positive (44, 38)
          1.0
            Negative (3, 17)
        end of Fare
    end of Pclass
  male
    Best: ('Pclass', 0.04147079049162472)
      1
        Best: ('Fare', 0.055316355424823827)
          0.0
            Negative (0, 4)
          1.0
            Negative (38, 57)
          2.0
            Positive (2, 0)
        end of Fare
      2
        Negative (15, 84)
      3
        Negative (38, 215)
    end of Pclass
end of Sex


**Salida esperada**
```js
Best: ('Sex', 0.21410831283572307)
  female
    Best: ('Pclass', 0.2267010508317432)
      1
        Positive (80, 3)
      2
        Positive (68, 6)
      3
        Best: ('Fare', 0.07516755751248805)
          0.0
            Positive (44, 38)
          1.0
            Negative (3, 17)
        end of Fare
    end of Pclass
  male
    Best: ('Pclass', 0.04147079049162472)
      1
        Best: ('Fare', 0.055316355424823827)
          0.0
            Negative (0, 4)
          1.0
            Negative (38, 57)
          2.0
            Positive (2, 0)
        end of Fare
      2
        Negative (15, 84)
      3
        Negative (38, 215)
    end of Pclass
end of Sex
```

**Felicitaciones**

Has completado de manera correcta la implementación de unas de las variaciones de los árboles de inducción. Por mantener el código simple, este ejemplo solo imprime el árbol a medida que lo contruye. Modifique el código para almacenar el resultado en un árbol de verdad